In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Carregar bibliotecas

In [ ]:
import pandas as pd
import glob

### Lendo o arquivo com os resultados de validação




In [ ]:
resultados_validacao = pd.read_csv('resultados_treino.csv', sep=';')

resultados_validacao.head(10)

,tecnica,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,identificador_aleatorio,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params
0,RandomForestRegressor,45.751296,4583.342971,0.029482,32.420000,5140650,train_test_split 25%,robust_scaler,Não utilizado,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",30,26,9,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': True, 'robust_..."
1,RandomForestRegressor,47.192311,4792.970416,0.037296,33.793333,4911162,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,3,3,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'auto..."
2,ExtraTreesRegressor,45.491102,4563.042866,0.026749,31.923333,2998007,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,1,6,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': False, 'pca__svd_solver': 'aut..."
3,BaggingRegressor,46.284192,4690.707759,0.018800,31.505000,2993473,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,42,6,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'auto..."
4,AdaBoostRegressor,60.368949,5627.104701,3.389285,41.016463,5525636,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,12,5,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': False, 'pca__svd_solver': 'aut..."
5,KNeighborsRegressor,49.217162,5429.718396,0.022046,33.071508,3105363,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,26,6,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': False, 'pca__svd_solver': 'ran..."
6,ExtraTreeRegressor,63.546868,9060.110783,0.049803,42.000000,2523323,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,42,4,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'auto..."
7,RandomForestRegressor,45.465008,4519.388870,0.036350,32.040000,5231028,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",28,5,9,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'rand..."
8,ExtraTreesRegressor,46.941264,4727.680226,0.036849,33.270000,3844330,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",28,19,6,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'auto..."
9,BaggingRegressor,46.349298,4781.590962,0.019576,31.060000,761450,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",28,15,4,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': False, 'pca__svd_solver': 'aut..."


##### Identificando as 15 melhores desempenhos com base na métrica 'mean_squared_log_error'

In [ ]:
resultados_validacao.sort_values('mean_squared_log_error', ascending=True).head(15)

,tecnica,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,identificador_aleatorio,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params
23,BaggingRegressor,45.356810,4471.776963,0.018556,31.350000,9386803,train_test_split 20%,robust_scaler,FastICA,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",9,20,9,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust..."
3,BaggingRegressor,46.284192,4690.707759,0.018800,31.505000,2993473,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,42,6,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'auto..."
65,BaggingRegressor,45.980528,4678.450657,0.019066,31.886667,9546218,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",28,13,3,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'full..."
108,BaggingRegressor,46.354920,4675.939457,0.019099,33.410000,2422492,train_test_split 25%,robust_scaler,pca,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",3,4,8,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust..."
112,KNeighborsRegressor,46.528189,4730.477725,0.019460,33.171429,3662094,train_test_split 25%,robust_scaler,pca,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",3,39,3,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': True, 'robust_..."
116,BaggingRegressor,47.124294,4813.432485,0.019480,32.630000,6166979,train_test_split 20%,min_max_scaler,Não utilizado,"[('min_max_scaler', MinMaxScaler(copy=True, fe...","{'min_max_scaler__feature_range': [(0, 1), (1,...",18,11,6,"Pipeline(memory=None,\n steps=[('min_m...","{'min_max_scaler__feature_range': (1, 100), 'B..."
9,BaggingRegressor,46.349298,4781.590962,0.019576,31.060000,761450,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",28,15,4,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': False, 'pca__svd_solver': 'aut..."
106,RandomForestRegressor,47.937317,4976.233836,0.020206,34.960000,2044671,train_test_split 25%,robust_scaler,pca,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",3,39,3,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust..."
5,KNeighborsRegressor,49.217162,5429.718396,0.022046,33.071508,3105363,train_test_split 20%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",24,26,6,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': False, 'pca__svd_solver': 'ran..."
54,RandomForestRegressor,52.711453,6081.238817,0.023795,37.406667,5669951,train_test_split 30%,normalizer,Não utilizado,"[('normalizer', Normalizer(copy=True, norm='l2...","{'normalizer__norm': ('l1', 'l2', 'max'), 'Ran...",18,33,8,"Pipeline(memory=None,\n steps=[('norma...","{'normalizer__norm': 'max', 'RandomForestRegre..."


In [ ]:
# Verificando a métrica 'mean_absolute_error'
resultados_validacao.sort_values('mean_absolute_error', ascending=True).head(15)

,tecnica,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,identificador_aleatorio,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params
26,KNeighborsRegressor,44.224415,4336.690380,0.028458,31.042389,5055416,train_test_split 20%,robust_scaler,FastICA,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",9,4,3,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust..."
46,KNeighborsRegressor,44.226949,4380.901824,0.048919,30.495711,2301037,train_test_split 30%,robust_scaler,pca,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",38,21,8,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust..."
35,RandomForestRegressor,44.632619,4317.044508,0.069584,30.980000,5980600,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",5,36,9,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'full..."
38,GradientBoostingRegressor,44.740314,4110.460758,1.163296,29.623820,1708492,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",5,40,7,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'auto..."
37,BaggingRegressor,45.047649,4413.699933,0.029008,32.460000,7009378,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",5,3,8,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'full..."
68,KNeighborsRegressor,45.202952,4485.138344,0.027614,31.777778,8194113,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",28,23,9,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': False, 'pca__svd_solver': 'aut..."
50,BaggingRegressor,45.252505,4554.759638,0.032423,31.233333,5842068,train_test_split 20%,standard_scaler,Não utilizado,"[('standard_scaler', StandardScaler(copy=True,...","{'standard_scaler__with_mean': [True, False], ...",42,14,9,"Pipeline(memory=None,\n steps=[('stand...","{'standard_scaler__with_std': True, 'standard_..."
79,RandomForestRegressor,45.284136,4483.494712,0.054886,31.726667,1047314,train_test_split 25%,standard_scaler,FastICA,"[('standard_scaler', StandardScaler(copy=True,...","{'standard_scaler__with_mean': [True, False], ...",42,20,4,"Pipeline(memory=None,\n steps=[('stand...","{'standard_scaler__with_std': False, 'standard..."
23,BaggingRegressor,45.356810,4471.776963,0.018556,31.350000,9386803,train_test_split 20%,robust_scaler,FastICA,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",9,20,9,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust..."
64,ExtraTreesRegressor,45.387327,4475.322279,0.035231,32.806667,9962607,train_test_split 25%,Não utilizado,pca,"[('pca', PCA(copy=True, iterated_power='auto',...","{'pca__n_components': [None, 3, 5, 7, 9, 11, 1...",28,24,10,"Pipeline(memory=None,\n steps=[('pca',...","{'pca__whiten': True, 'pca__svd_solver': 'full..."


***

# Retreinar melhor modelo com os dados completos, sem split

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import FastICA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
import sklearn
import joblib
import ast

In [ ]:
melhor_modelo = resultados_validacao.sort_values('mean_absolute_error', ascending=True).head(1)
melhor_modelo

,tecnica,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,identificador_aleatorio,tipo_split,normalizador,redutor_dimensionalidade,pipeline,parametros,random_state_split,random_state_random_search,cross_validation_random_search,best_estimator,best_params
26,KNeighborsRegressor,44.224415,4336.69038,0.028458,31.042389,5055416,train_test_split 20%,robust_scaler,FastICA,"[('robust_scaler', RobustScaler(copy=True, qua...","{'robust_scaler__with_centering': [True, False...",9,4,3,"Pipeline(memory=None,\n steps=[('robus...","{'robust_scaler__with_scaling': False, 'robust..."


In [ ]:
pipeline = melhor_modelo['pipeline'].values[0]
pipeline = Pipeline([('standard_scaler', StandardScaler()), ('GradientBoostingRegressor', GradientBoostingRegressor())])
pipeline

Pipeline(memory=None,
         steps=[('standard_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('GradientBoostingRegressor',
                 GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                           criterion='friedman_mse', init=None,
                                           learning_rate=0.1, loss='ls',
                                           max_depth=3, max_features=None,
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators=100,
                                           n_iter_no_change=

In [ ]:
#parametros = melhor_modelo['parametros'].values[0]
#parametros = ast.literal_eval(parametros)
#parametros
parametros = {
            'standard_scaler__with_mean': [True, False],
            'standard_scaler__with_std': [True, False],
            'GradientBoostingRegressor__loss': ('ls', 'lad', 'huber', 'quantile'),
            'GradientBoostingRegressor__criterion': ('friedman_mse', 'mse', 'mae'),
            'GradientBoostingRegressor__max_features': ('auto', 'sqrt', 'log2'),
            'GradientBoostingRegressor__n_estimators': [50, 100, 150],
            'GradientBoostingRegressor__warm_start': [True, False]
        }
parametros

{'GradientBoostingRegressor__criterion': ('friedman_mse', 'mse', 'mae'),
 'GradientBoostingRegressor__loss': ('ls', 'lad', 'huber', 'quantile'),
 'GradientBoostingRegressor__max_features': ('auto', 'sqrt', 'log2'),
 'GradientBoostingRegressor__n_estimators': [50, 100, 150],
 'GradientBoostingRegressor__warm_start': [True, False],
 'standard_scaler__with_mean': [True, False],
 'standard_scaler__with_std': [True, False]}

In [ ]:
cv = melhor_modelo['cross_validation_random_search'].values[0]
cv

3

In [ ]:
random_state = melhor_modelo['random_state_random_search'].values[0]
random_state

4

In [ ]:
# Ler dados de train completo
train = pd.read_csv('dados/train_tratado-3_2.csv', sep=';')
train.drop(columns=['NU_INSCRICAO'], inplace=True)
X = train.drop(columns=['NU_NOTA_MT'])
y = train['NU_NOTA_MT']

In [ ]:
#modelo = GridSearchCV(estimator=pipeline, param_grid=parametros, cv=cv, n_jobs= -1, verbose=2)
modelo = RandomizedSearchCV(n_iter=10, estimator=pipeline, param_distributions=parametros, cv=5, random_state=1, n_jobs= -1, verbose=42)

In [ ]:
modelo.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 27.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 27

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('standard_scaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('GradientBoostingRegressor',
                                              GradientBoostingRegressor(alpha=0.9,
                                                                        ccp_alpha=0.0,
                                                                        criterion='friedman_mse',
                                                                        init=None,
                                                                        learning_rate=0.1,
                                                                        loss='ls',
               

In [ ]:
nome_arquivo = 'modelos/melhor/GradientBoostingRegressor.pkl'
joblib.dump(modelo.best_estimator_, nome_arquivo)

['drive/My Drive/Estudos/codenation/modelos/melhor/GradientBoostingRegressor.pkl']